<a href="https://colab.research.google.com/github/moqri/deep_cell/blob/master/starter_code/blood_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#install required packages
!pip install scanpy

In [16]:
# Download the data 
!mkdir adultPancreas
!cd adultPancreas
!wget https://cells.ucsc.edu/adultPancreas/exprMatrix.tsv.gz
!wget https://cells.ucsc.edu/adultPancreas/meta.tsv

mkdir: cannot create directory ‘adultPancreas’: File exists
--2020-01-22 21:09:51--  https://cells.ucsc.edu/adultPancreas/exprMatrix.tsv.gz
Resolving cells.ucsc.edu (cells.ucsc.edu)... 128.114.119.131, 128.114.119.132
Connecting to cells.ucsc.edu (cells.ucsc.edu)|128.114.119.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22707068 (22M) [application/x-gzip]
Saving to: ‘exprMatrix.tsv.gz.1’

exprMatrix.tsv.gz.1 100%[===================>]  21.65M  9.13MB/s    in 2.4s    

2020-01-22 21:09:54 (9.13 MB/s) - ‘exprMatrix.tsv.gz.1’ saved [22707068/22707068]

--2020-01-22 21:09:56--  https://cells.ucsc.edu/adultPancreas/meta.tsv
Resolving cells.ucsc.edu (cells.ucsc.edu)... 128.114.119.131, 128.114.119.132
Connecting to cells.ucsc.edu (cells.ucsc.edu)|128.114.119.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 522326 (510K) [text/tab-separated-values]
Saving to: ‘meta.tsv.1’

meta.tsv.1          100%[===================>] 510.08K   604

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
import itertools
import numpy as np
import scanpy as sc
import pandas as pd

In [23]:
ad = sc.read_text("exprMatrix.tsv.gz")
meta = pd.read_csv("meta.tsv", sep="\t")
ad.var = meta
ad.shape

(23450, 4026)

In [33]:
ad.X

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   3.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [108.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [25]:
expression_df=pd.DataFrame(ad.X).transpose()
expression_df.shape

(4026, 23450)

In [26]:
expression_df.index.rename('Cell_ID',inplace=1)
expression_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,23410,23411,23412,23413,23414,23415,23416,23417,23418,23419,23420,23421,23422,23423,23424,23425,23426,23427,23428,23429,23430,23431,23432,23433,23434,23435,23436,23437,23438,23439,23440,23441,23442,23443,23444,23445,23446,23447,23448,23449
Cell_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,12.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,108.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,148.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,124.0,12.0,0.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,164.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,230.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,115.0,0.0,454.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
cell_types=meta.cluster
cell_df=pd.DataFrame(cell_types)
cell_df.index.rename('Cell_ID',inplace=1)
cell_df.columns=['Cell_Type']
cell_df.dropna(inplace=True)
cell_df.head()

,Cell_Type
Cell_ID,
0,unsure
1,alpha
2,unsure
4,delta
5,delta


In [67]:
cell_df['Cell_Type'].value_counts()

alpha          998
acinar         411
ductal         389
beta           347
unsure         262
delta           82
mesenchymal     53
Name: Cell_Type, dtype: int64

In [0]:
expression_df=expression_df.loc[cell_df.index]
cell_df['Cell_Type'] = pd.Categorical(cell_df['Cell_Type'])
cell_df['Cell_Type'] = cell_df['Cell_Type'].cat.codes

In [0]:
expression_array=expression_df.values
expression_array=expression_array[:,~np.all(expression_array == 0, axis=0)] # remove all zero columns
expression_array_log=np.log(expression_array+1)
cell_array=cell_df.values
number_of_genes_top_keep=1000
expression_array_top=SelectKBest(chi2, k=number_of_genes_top_keep).fit_transform(expression_array_log, cell_array)
expression_array_normalized=np.round(expression_array_top) 
#(expression_array_top / np.linalg.norm(expression_array_top),3)

In [86]:
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(
    expression_array_normalized, cell_array, test_size=0.33, random_state=1)
clf = svm.SVC(decision_function_shape='ovo')
y_predict=clf.fit(X_train, y_train)
y_predict=clf.predict(X_test)
accuracy_score(y_test, y_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9046483909415971

In [87]:
predictions=pd.DataFrame(data={'label':list(itertools.chain(*y_test)),'predicted':list(y_predict)})
predictions.head()

,label,predicted
0,4,4
1,1,1
2,0,0
3,2,2
4,1,1


In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [0]:
tf.random.set_seed(1)

In [90]:
dataset=tf.data.Dataset.from_tensor_slices((expression_array_normalized,cell_array ))
dataset

<TensorSliceDataset shapes: ((10000,), (1,)), types: (tf.float32, tf.int8)>

In [91]:
for feat, targ in dataset.take(1):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0. ... 0. 0. 5.], Target: [6]


In [0]:
train_dataset = dataset.shuffle(len(cell_df)).batch(1)
cell_type_count=len(cell_df['Cell_Type'].value_counts())

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(cell_type_count, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [102]:
model = get_compiled_model()
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test,  y_test, verbose=2)

Train on 1703 samples
Epoch 1/10
1703/1703 [==============================] - 0s 217us/sample - loss: 0.9989 - accuracy: 0.8021
Epoch 2/10
1703/1703 [==============================] - 0s 98us/sample - loss: 0.1973 - accuracy: 0.9419
Epoch 3/10
1703/1703 [==============================] - 0s 101us/sample - loss: 0.0813 - accuracy: 0.9794
Epoch 4/10
1703/1703 [==============================] - 0s 96us/sample - loss: 0.0562 - accuracy: 0.9865
Epoch 5/10
1703/1703 [==============================] - 0s 91us/sample - loss: 0.0481 - accuracy: 0.9877
Epoch 6/10
1703/1703 [==============================] - 0s 92us/sample - loss: 0.0454 - accuracy: 0.9877
Epoch 7/10
1703/1703 [==============================] - 0s 92us/sample - loss: 0.0436 - accuracy: 0.9877
Epoch 8/10
1703/1703 [==============================] - 0s 105us/sample - loss: 0.0422 - accuracy: 0.9877
Epoch 9/10
1703/1703 [==============================] - 0s 109us/sample - loss: 0.0409 - accuracy: 0.9877
Epoch 10/10
1703/1703 [======

[0.34026898538898587, 0.91775924]